In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.metrics.aline import similarity_matrix
from pandas import value_counts
from win32evtlogutil import langid

- kaggle dataset (초안)

In [2]:
book_df=pd.read_csv('book_review_data.csv')

In [3]:
# 1) 필요한 컬럼 도출
# 2) 형변환 (float > int)

book_df=book_df[['user_id',"age","rating","book_title",'book_author','year_of_publication','img_m','Summary','Category','country','Language']]
book_df[['age',"year_of_publication"]]=book_df[['age',"year_of_publication"]].astype(int)

In [21]:
# 최소 평가 개수 ( 평가가 30개보다 많은 책만 추출 )
counts=book_df['book_title'].value_counts()
title = counts[counts > 30].index
book_filter_df=book_df[book_df['book_title'].isin(title)]

In [22]:
# 최소 평가 개수 ( 평가가 20개보다 많은 사용자만 추출 )
counts=book_filter_df["user_id"].value_counts()
user_id_=counts[counts>10].index
book_filter_df = book_filter_df[book_filter_df['user_id'].isin(user_id_)]

In [23]:
# 무의미한 0점 평가 제거
rating_zero=book_filter_df[book_filter_df['rating']==0].index
book_filter_df.drop(rating_zero, axis=0, inplace=True)

In [24]:
# 한 카테고리안에 책이 5권 미만인 경우 제거
counts=book_filter_df["Category"].value_counts()
categories = counts[counts > 5].index
book_filter_df = book_filter_df[book_filter_df['Category'].isin(categories)]
idx=book_filter_df[book_filter_df["Summary"]==9].index
book_filter_df.drop(idx,axis=0,inplace=True)

In [25]:
book_filter_df.shape
len(book_filter_df['user_id'].unique())

4920

In [26]:
user_book_df=book_filter_df.pivot_table(index='user_id',columns='book_title',values='rating',fill_value=0)
user_book_df.shape # user간 유사도 구할거 아니니까
book_user_df=user_book_df.T

In [27]:
user_book_df.iloc[55].sort_values(ascending=False)[:10]

book_title
It                                                                          8.0
Buddha of Suburbia                                                          8.0
Christine                                                                   8.0
Thinner                                                                     8.0
Firestarter (Signet Book)                                                   7.0
Practical Demonkeeping                                                      7.0
Dolores Claiborne                                                           7.0
Something Wicked This Way Comes                                             6.0
Cujo                                                                        6.0
Stupid White Men ...and Other Sorry Excuses for the State of the Nation!    6.0
Name: 4334, dtype: float64

In [28]:
from sklearn.metrics.pairwise import cosine_similarity
book_similarity=cosine_similarity(book_user_df,book_user_df)
book_similarity_df=pd.DataFrame(book_similarity)

In [30]:
book_filter_df.to_csv("book_filter",index=False, encoding='utf-8')